In [1]:
ACTORS = ['Chandler', 'Joey', 'Monica', 'Phoebe', 'Rachel', 'Ross']
DATASET_PATH = '/home/tk/datasets/MELD/visual-features/MELD-visual-features/datasets.json'

VIDS_DIR = {}
VIDS_DIR['train'] = "/home/tk/datasets/MELD/MELD.Raw/train/train_splits/" 
VIDS_DIR['dev'] = "/home/tk/datasets/MELD/MELD.Raw/dev/dev_splits_complete/" 
VIDS_DIR['test'] = "/home/tk/datasets/MELD/MELD.Raw/test/output_repeated_splits_test/"

SAVE_AT = "/home/tk/datasets/MELD/visual-features/MELD-visual-features/videos/"
VID2ANNO_PATH = "/home/tk/datasets/MELD/visual-features/MELD-visual-features/datasets.json"

VISUAL_FEATURES_DIR = {}
VISUAL_FEATURES_DIR['train'] = '/home/tk/datasets/MELD/visual-features/MELD-visual-features/train/'
VISUAL_FEATURES_DIR['dev'] = '/home/tk/datasets/MELD/visual-features/MELD-visual-features/dev/'
VISUAL_FEATURES_DIR['test'] = '/home/tk/datasets/MELD/visual-features/MELD-visual-features/test/'

FACE_EMBEDDINGS_DIR = './main-actors/'

ANGLE_DIST_THRESHOLD = 1.15

In [2]:
import numpy as np
import os
from glob import glob
import csv
import json
from tqdm.notebook import tqdm

with open(VID2ANNO_PATH, 'r', encoding='utf8') as stream:
    vid2anno = json.load(stream)

face_embeddings = {os.path.basename(filename).split('.npy')[0]: np.load(filename).reshape(1, 512) for filename in glob(os.path.join(FACE_EMBEDDINGS_DIR, '*.npy'))}

MAIN_ACTORS = sorted([foo.split('.npy')[0] for foo in os.listdir(FACE_EMBEDDINGS_DIR)])

In [3]:
from cltl_face_all.face_alignment import FaceDetection
import av
from av import InvalidDataError
from cltl_face_all.arcface import calc_angle_distance

fd = FaceDetection(device='cpu', face_detector='sfd')

for DATATYPE in tqdm(['train', 'dev', 'test']):
    os.makedirs(os.path.join(SAVE_AT, DATATYPE), exist_ok=True)

    for diautt, anno in tqdm(vid2anno[DATATYPE].items()):
        try:
            speaker = vid2anno[DATATYPE][diautt]['Speaker']
        except KeyError as e:
            print(f"{diautt} not in {DATATYPE}, {e}")

        if speaker not in MAIN_ACTORS:
            continue

        vidpath = os.path.join(VIDS_DIR[DATATYPE], diautt) + '.mp4'
        try:
            container = av.open(vidpath)
            fps = round(float(container.streams.video[0].average_rate))
        except InvalidDataError as e:
            print(e)
            continue

        try:
            features = np.load(os.path.join(VISUAL_FEATURES_DIR[DATATYPE], 
                                f'{diautt}.npy'), allow_pickle=True).item()
        except Exception as e:
            print(f"Couldn't load the features {e}")

        faces = []
        for idx, frame in enumerate(container.decode(video=0)):
            feature = features[idx]

            if len(feature) == 0:
                continue

            numpy_RGB = np.array(frame.to_image())
            batch = numpy_RGB[np.newaxis, ...]

            for feat in feature:
                bbox = feat['bbox']
                landmark = feat['landmark']
                age = feat['age']
                gender = feat['gender']
                embedding = feat['embedding']
                embedding = embedding.reshape(1, 512)

                dist = calc_angle_distance(embedding, face_embeddings[speaker])
                dist = float(dist[0, 0])

                if dist >= ANGLE_DIST_THRESHOLD:
                    continue

                face = fd.crop_and_align(batch, [bbox[np.newaxis, ...]], [landmark[np.newaxis, ...]])
                face = np.squeeze(face)
                faces.append(face)

        if len(faces) == 0:
            continue
    
        save_full_path = os.path.join(SAVE_AT, DATATYPE, diautt) + '.mp4'
        container = av.open(save_full_path, mode='w')

        stream = container.add_stream('mpeg4', rate=fps)
        stream.width = 112
        stream.height = 112

        for face in faces:
            frame = av.VideoFrame.from_ndarray(face, format='rgb24')
            for packet in stream.encode(frame):
                container.mux(packet)
        # Flush stream
        for packet in stream.encode():
            container.mux(packet)

        # Close the file
        container.close()

o 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too man

o 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too man

 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many threads/slices (8), reducing to 7
too many